<a href="https://colab.research.google.com/github/EdmilsonSantana/llm-vehicle-repair/blob/main/Assistente_do_Mecanico_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
%%capture
!pip install docx

In [125]:
from docx import Document
from docx.text.paragraph import Paragraph
from docx.shared import Pt
import os
import re
import modal
import json

### Processando apostilas da Universidade do Automóvel

In [109]:
RAW_DIR = './raw_data'
BASE_DIR = './data'

In [110]:
files = os.listdir(RAW_DIR)

In [111]:
print(f'{len(files)} documents')

4 documents


In [112]:
files

['5 -Motor de Comb. Interna - Otto.docx',
 '6 - Sistema de Transmissão.docx',
 '7 - Metrologia Mec. Automotiva.docx',
 '8 - Suspensão, Direção e Freios.docx']

In [113]:
document = Document(f'{RAW_DIR}/{files[0]}')

In [114]:
def has_small_font(paragraph: Paragraph):
    font_size = paragraph.style.font.size
    return font_size == Pt(9) or font_size == Pt(8)

In [115]:
def is_heading(paragraph: Paragraph):
    return paragraph.style.name.startswith('Heading')


In [116]:
def parse_text(text):
    text = re.sub('\s+',' ',text).strip()
    return '' if text.isnumeric() else text

In [131]:
doc_content = {}
curr_heading = "TITLE"
last_text = curr_heading

for section in document.sections:
    header_paragraphs = section.header.paragraphs
    footer_paragraphs = section.footer.paragraphs

    for item in section.iter_inner_content():
        if (not isinstance(item, Paragraph)):
            continue

        text = parse_text(item.text)
        if (len(text) == 0):
            continue
        if (item in header_paragraphs or item in footer_paragraphs):
            continue
        if (has_small_font(item)):
            continue

        if (is_heading(item)):
            curr_heading = text
        
        section_content = doc_content.setdefault(curr_heading, [])
        if text != curr_heading:
            section_content.append(text)
            

In [132]:
doc_content.keys()

dict_keys(['TITLE', 'CICLO OTTO', 'Conteúdos', 'INTRODUÇÃO', 'MOTOR DE COMBUSTÃO', 'MOTOR DE COMBUSTÃO EXTERNA', 'MOTOR DE COMBUSTÃO INTERNA', 'MOTORES DE DOIS TEMPOS', 'MOTORES DE QUATRO TEMPOS', 'CABEÇOTE DO MOTOR', 'TIPOS DE CABEÇOTE', 'DISTRIBUIÇÃO MECÂNICA', 'DIAGRAMA DE VÁLVULAS', 'POSIÇÃO DO COMANDO E TIPO DE MOTOR', 'ESTRUTURA DA DISTRIBUIÇÃO MECÂNICA', 'ÁRVORE DE COMANDO DE VÁLVULAS', 'TUCHOS', 'CONJUNTO DE BALANCINS', 'MANUTENÇÃO', 'VÁLVULAS', 'CONSTITUIÇÃO DA VÁLVULA', 'DISPOSITIVOS DE MONTAGEM DAS VÁLVULAS', 'PEÇAS MÓVEIS', 'BIELA', 'CASQUILHO', 'ÊMBOLO', 'ANÉIS DE SEGMENTO', 'ÁRVORE DE MANIVELAS', 'BLOCO DE MOTOR', 'TIPOS DE BLOCO', 'VOLANTE DO MOTOR', 'FUNÇÕES DO VOLANTE DO MOTOR', 'DIMENSÕES E DESEMPENHO DO MOTOR', 'LUBRIFICAÇÃO', 'ATRITO', 'LUBRIFICANTES', 'SISTEMA DE LUBRIFICAÇÃO', 'SISTEMA DE LUBRIFICAÇÃO MISTO', 'SISTEMA DE CÁRTER SECO', 'CÁRTER', 'FILTRO DE ÓLEO', 'BOMBA DE ÓLEO', 'VÁLVULA REGULADORA DE PRESSÃO', 'BIBLIOGRAFIA'])

In [133]:
doc_content.pop('TITLE')
doc_content.pop('CICLO OTTO')
doc_content.pop('Conteúdos')
doc_content.pop('INTRODUÇÃO')
doc_content.pop('BIBLIOGRAFIA')

['https://www.mmcofap.com.br',
 'CASTROL. Veículos automotivos e sua lubrificação.',
 'COFAP. Manual doutor em motores. 2a.ed., 1988.',
 'FIAT. Oficina FIAT no. 2, vol. II, ano2. 1987. Tutela lubrificantes: história, informações, técnicas e Produtos.',
 'MOBIL. Curso de lubrificação. São Paulo, 1975.']

In [134]:
doc_content.keys()

dict_keys(['MOTOR DE COMBUSTÃO', 'MOTOR DE COMBUSTÃO EXTERNA', 'MOTOR DE COMBUSTÃO INTERNA', 'MOTORES DE DOIS TEMPOS', 'MOTORES DE QUATRO TEMPOS', 'CABEÇOTE DO MOTOR', 'TIPOS DE CABEÇOTE', 'DISTRIBUIÇÃO MECÂNICA', 'DIAGRAMA DE VÁLVULAS', 'POSIÇÃO DO COMANDO E TIPO DE MOTOR', 'ESTRUTURA DA DISTRIBUIÇÃO MECÂNICA', 'ÁRVORE DE COMANDO DE VÁLVULAS', 'TUCHOS', 'CONJUNTO DE BALANCINS', 'MANUTENÇÃO', 'VÁLVULAS', 'CONSTITUIÇÃO DA VÁLVULA', 'DISPOSITIVOS DE MONTAGEM DAS VÁLVULAS', 'PEÇAS MÓVEIS', 'BIELA', 'CASQUILHO', 'ÊMBOLO', 'ANÉIS DE SEGMENTO', 'ÁRVORE DE MANIVELAS', 'BLOCO DE MOTOR', 'TIPOS DE BLOCO', 'VOLANTE DO MOTOR', 'FUNÇÕES DO VOLANTE DO MOTOR', 'DIMENSÕES E DESEMPENHO DO MOTOR', 'LUBRIFICAÇÃO', 'ATRITO', 'LUBRIFICANTES', 'SISTEMA DE LUBRIFICAÇÃO', 'SISTEMA DE LUBRIFICAÇÃO MISTO', 'SISTEMA DE CÁRTER SECO', 'CÁRTER', 'FILTRO DE ÓLEO', 'BOMBA DE ÓLEO', 'VÁLVULA REGULADORA DE PRESSÃO'])

### Extraindo instruções a partir do conteúdo

In [164]:
! modal deploy inference.py

- Creating objects...
| Creating objects...
└── - Creating mount e:\Workspace\tcc-2022-2\inference.py: Uploaded 0/0 
    inspected files
- Creating objects...
└── | Creating mount e:\Workspace\tcc-2022-2\inference.py: Uploaded 0/0 
    inspected files
| Creating objects...
└── - Creating mount e:\Workspace\tcc-2022-2\inference.py: Building mount
- Creating objects...
└── 🔨 Created mount e:\Workspace\tcc-2022-2\inference.py
| Creating objects...
└── 🔨 Created mount e:\Workspace\tcc-2022-2\inference.py
- Creating objects...
└── 🔨 Created mount e:\Workspace\tcc-2022-2\inference.py
| Creating objects...
├── 🔨 Created mount e:\Workspace\tcc-2022-2\inference.py
└── - Creating download_model_to_folder...
- Creating objects...
├── 🔨 Created mount e:\Workspace\tcc-2022-2\inference.py
└── 🔨 Created download_model_to_folder.
| Creating objects...
├── 🔨 Created mount e:\Workspace\tcc-2022-2\inference.py
└── 🔨 Created download_model_to_folder.
- Creating objects...
├── 🔨 Created mount e:\Workspace\

In [165]:
inference = modal.Function.lookup('llm-inference', 'Model.generate')

#### Prompts

In [192]:
questions_prompt = """
You are a specialist in vehicle repair and maintenance and your goal is to extract questions and answers related to a certain topic from a document.
Here are some criterias that you have to follow to achieve your goal:

- You should not create questions for contents not related to vehicle repair and maintenance.
- You should include the topic in the question so that it is possible to understand what the question is about.
- The answers should only contain the content originating from the document.
- You must create a maximum of 5 questions.
- The language used to generate questions and answers should always be Brazilian Portuguese.
- Your response should always be only a well-formed JSON array with the questions and answers.

examples:

Topic: Rolamentos dianteiros
Document: O cuidado com o meio ambiente é uma preocupação global imprescindível
para o desenvolvimento sustentável. No mundo dos automóveis, essa discussão é cada vez maior,
com o aumento do rigor na legislação em diversos países - o que acelera a evolução de veículos de propulsão alternativa,
como híbridos e elétricos. O aperto excessivo da porca afeta diretamente a vida útil do rolamento.
Em rolamentos dianteiros selados, Airton, da Schaeffler, adverte para nunca
substituir a graxa ou completar os espaços internos, porque pode haver uma reação
entre as graxas com composições químicas diferentes, além de um aquecimento elevado no interior do rolamento.

Answer:
[
    {{
        "question": "Posso apertar a porca do rolamento ?",
        "answer": "O aperto excessivo da porca afeta diretamente a vida útil do rolamento."
    }},
    {{
        "question": "Devo substituir a graxa de rolamentos dianteiros selados ? ",
        "answer": "Em rolamentos dianteiros selados, nunca substitua a graxa ou complete os espaços internos,
                porque pode haver uma reação entre as graxas com composições químicas diferentes, além de
                um aquecimento elevado no interior do rolamento."
    }}
]

Topic: {topic}
Document: {document}
Answer:"""

In [167]:
topics_prompt = """
Given a document your goals is to extract 5 topics that summarizes the content present in the document.

examples:

Document: O cuidado com o meio ambiente é uma preocupação global imprescindível
para o desenvolvimento sustentável. No mundo dos automóveis, essa discussão é cada vez maior,
com o aumento do rigor na legislação em diversos países - o que acelera a evolução de veículos de propulsão alternativa,
como híbridos e elétricos. O aperto excessivo da porca afeta diretamente a vida útil do rolamento.
Em rolamentos dianteiros selados, Airton, da Schaeffler, adverte para nunca
substituir a graxa ou completar os espaços internos, porque pode haver uma reação
entre as graxas com composições químicas diferentes, além de um aquecimento elevado no interior do rolamento.

Answer:
[
    "A importância global do cuidado com o meio ambiente",
    "A evolução dos veículos de propulsão alternativa",
    "O aumento da legislação ambiental nos países",
    "Impacto do aperto excessivo da porca na vida útil do rolamento",
    "Cuidados específicos com a manutenção de rolamentos dianteiros selados"
]

Document: {document}
Answer:"""

In [161]:
documents = []
for sentences in doc_content.values():
    documents.append(" ".join(sentences))

In [162]:
documents[0]

'Há muitos séculos, o homem vem se defrontando com problemas de transporte; nos primórdios da civilização foi utilizada a tração animal, porém muito dos esforços humanos foram dirigidos à construção de veículos que não dependessem da tração animal. Um dos primeiros resultados foi a invenção da máquina a vapor, conhecida também como motor de combustão externa, logo em seguida a sua invenção, foi instalada nos veículos já existentes (carruagens, triciclos e bicicletas). Consta que o primeiro desses veículos, um triciclo a vapor, foi construído na França, já em 1771. Esse triciclo iniciou um processo que não se interrompeu mais; a produção de veículos que têm propulsão própria. A máquina a vapor, entretanto, apresenta muitos problemas e inconvenientes: Seu baixo rendimento obriga-a a utilizar grandes quantidades de combustível e água; A produção de vapor em caldeiras, pelo calor gerado nas fornalhas, obriga o motor a ocupar muito espaço. Por isso, torna-se difícil fazer um carro a vapor c

In [176]:
def create_topics(dataset, documents):
    failed_responses = []
    for i, document in enumerate(documents):
        try:
            print(f"Processing document with {len(document)} length")
            response = inference.remote(topics_prompt.format(document=document))
            data = dataset.setdefault(i, {})
            data.update({'topics': json.loads(response)})
        except Exception as error:
            print(f"{i=} {error=}")
            failed_responses.append(i)

    return failed_responses

In [193]:
def get_questions(dataset, documents):
    failed_responses = []
    for i, data in dataset.items():
        document = documents[i]
        for topic in data['topics']:
            try:
                print(f"Processing topic \"{topic}\" from document {i}")
                prompt = questions_prompt.format(topic=topic, document=document)
                response = inference.remote(prompt)
                questions = data.setdefault("questions", [])
                questions.extend(json.loads(response))
            except Exception as error:
                print(f"{i=} {topic=} {error=}")
                failed_responses.append((i, topic))
    return failed_responses

In [177]:
dataset = {}

In [178]:
failed_responses = create_topics(dataset, documents)
failed_responses

Processing document with 2529 length
Processing document with 607 length
Processing document with 3335 length
Processing document with 1536 length
Processing document with 1806 length
Processing document with 3697 length
Processing document with 355 length
Processing document with 2092 length
Processing document with 244 length
Processing document with 1265 length
Processing document with 1250 length
Processing document with 2153 length
Processing document with 1429 length
Processing document with 2097 length
Processing document with 675 length
Processing document with 1317 length
Processing document with 1319 length
Processing document with 1433 length
Processing document with 484 length
Processing document with 904 length
Processing document with 1593 length
Processing document with 3760 length
Processing document with 1349 length
Processing document with 2683 length
Processing document with 1681 length
Processing document with 397 length
Processing document with 1081 length
Processi

[]

In [194]:
failed_responses = get_questions(dataset, documents)
failed_responses

Processing topic Problemas de transporte ao longo da história da humanidade from document 0
Processing topic Invenção da máquina a vapor e sua utilização em veículos from document 0
Processing topic Desenvolvimento do motor de combustão externa e seus problemas from document 0
Processing topic Invento do motor de combustão interna do ciclo Otto from document 0
Processing topic Construção do primeiro automóvel com motor de combustão interna from document 0
Processing topic Funcionamento da locomotiva a vapor from document 1
Processing topic Fontes de combustível para locomotivas a vapor from document 1
Processing topic Utilização do calor para aquecer água e produzir vapor from document 1
Processing topic Mecanismo de movimento das locomotivas a vapor from document 1
Processing topic Origem dos primeiros triciclos a vapor baseados no princípio da locomotiva a vapor from document 1
Processing topic Motor de combustão interna e seu funcionamento from document 2
Processing topic Diversidad

[]

In [198]:
with open(f'{BASE_DIR}/questions_01.json', 'w', encoding='utf-8') as fp:
    json.dump(dataset, fp, ensure_ascii=False)

In [201]:
with open(f'{BASE_DIR}/raw_documents_01.json', 'w', encoding='utf-8') as fp:
    json.dump(doc_content, fp, ensure_ascii=False)

In [202]:
with open(f'{BASE_DIR}/documents_01.json', 'w', encoding='utf-8') as fp:
    json.dump(documents, fp, ensure_ascii=False)

### Metrics

In [34]:
! pip install deepeval


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
from deepeval.models.base_model import DeepEvalBaseLLM
from inference import BASE_MODEL
        
class LocalEvaluationModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model
    
    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        return self.model.remote(prompt)

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return BASE_MODEL

In [58]:
question = queries[0]['question']
answer = queries[0]['answer']
question, answer

('O que é o motor de combustão externa na locomotiva a vapor?',
 'O motor de combustão externa é o que movimenta a locomotiva a vapor, pois a queima do combustível ocorre fora dos compartimentos que produzem o movimento (cilindros).')

In [60]:
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase

model = LocalEvaluationModel(model=inference)

context=[document]

test_case = LLMTestCase(
    input=question,
    actual_output="O motor de combustão externa vai explodir",
    context=context
)
metric = HallucinationMetric(model=model, threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

# or evaluate test cases in bulk
evaluate([test_case], [metric])

1.0
The hallucination score is 1.00 because the actual output is unrelated to the context, as it discusses an external combustion engine explosion instead of the functioning of a steam locomotive, which is the topic of the context. This disconnection between the actual output and the context indicates a high degree of hallucination.
Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...




Metrics Summary

  - ❌ Hallucination (score: 1.0, threshold: 0.5, strict: False, evaluation model: internlm/internlm2-chat-7b, reason: The hallucination score is 1.00 because the actual output fails to provide a comprehensive explanation of the context. The context discusses the functioning of a steam locomotive and its reliance on an external combustion engine, while the actual output only mentions the explosion of the combustion engine, which is not an accurate representation of the context. This discrepancy indicates a significant deviation from the context, leading to a high hallucination score., error: None)

For test case:

  - input: O que é o motor de combustão externa na locomotiva a vapor?
  - actual output: O motor de combustão externa vai explodir
  - expected output: None
  - context: ['MOTOR DE COMBUSTÃO EXTERNA Na locomotiva a vapor, o combustível é o carvão ou a lenha. O calor produzido é utilizado para aquecer água em uma caldeira, transformando-a em vapor. O vapor s

✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

[TestResult(success=False, metrics=[<deepeval.metrics.hallucination.hallucination.HallucinationMetric object at 0x000002E652FEEA50>], input='O que é o motor de combustão externa na locomotiva a vapor?', actual_output='O motor de combustão externa vai explodir', expected_output=None, context=['MOTOR DE COMBUSTÃO EXTERNA Na locomotiva a vapor, o combustível é o carvão ou a lenha. O calor produzido é utilizado para aquecer água em uma caldeira, transformando-a em vapor. O vapor se expande e, com sua pressão, vai movimentar os êmbolos que acionam as rodas motrizes da locomotiva. fornal A locomotiva a vapor é movida por um motor de combustão externa, pois a queima do combustível ocorre fora dos compartimentos que produzem o movimento (cilindros). Seguindo o princípio de funcionamento da locomotiva a vapor, foram desenvolvidos os primeiros triciclos a vapor, como o do francês Cugnot. Veja representado na figura a seguir.'], retrieval_context=None)]